In [1]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import geopandas as gpd
import glob
from tqdm import tqdm

%matplotlib inline

In [22]:
data = pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\ngdbrock-fUS56\ngdbrock-fUS56\xtbIcpaesChem.csv').fillna(0)
ref = pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\ngdbrock-fUS56\tblRockGeoData.csv')

C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (12,14,26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
huc12=gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp') #reads in the hydraulic basin units and adds an area column
huc12= huc12.to_crs({'init': 'epsg:3732'})

In [55]:

from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona
#writes the point data to a shapefile in the dir called data.shp
geometry = [Point(xy) for xy in zip(ref.longitude, ref.latitude)]
crs = {'init': 'epsg:3732'} 
geo_df = GeoDataFrame(ref, crs={'init': 'epsg:4326'}, geometry=geometry)
#geo_df.to_file(driver='ESRI Shapefile', filename='data.shp')
projGeoDF=geo_df.to_crs(crs)
val_huc = gpd.sjoin(huc12, projGeoDF)

In [56]:
val_huc = gpd.sjoin(huc12, projGeoDF)

In [57]:
val_huc = val_huc.merge(data, on='lab_id') #this is the chemical data joined with the HUC dataset

In [58]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'
allFiles=glob.glob(path+"*.csv")
frame=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file, index_col=[0])
    listed.append(df)
frame = pd.concat(listed)
frame['HUC_12']=frame.HUC_12.astype(str)
CaptainHucstable = huc12.merge(frame, on='HUC_12' )

In [78]:
true_vals = []
for samples in tqdm(range(len(val_huc))):
    true_vals.append(val_huc.loc[samples].HUC_12)

bflat = []  
for samples in tqdm(range(4)):#len(val_huc))):
    formation = val_huc.loc[samples].stratgrphy[0:10]
    b = []
    for element in range(58,113):
        ppm = val_huc.columns[element] #select each element in the list
        valued = val_huc.loc[samples][element] #select each elements values 
        #from the big list, select element, formation, and concentration greater than the sample has, and create a list of the
        #basins that we should explore
        b0 = CaptainHucstable[(CaptainHucstable['element']== ppm) & (CaptainHucstable['formation'].str.contains(formation)) &(
            CaptainHucstable['bins_ppm']>= valued) & (CaptainHucstable['prob']>0.1)].HUC_12.values
        b.append(b0)
    bflat.append(np.unique(np.asarray([item for sublist in b for item in sublist])))
preds = np.unique(np.asarray([item for sublist in bflat for item in sublist]))
TP = len(set(true_vals).intersection(preds))
FP = len(preds)-len(set(true_vals).intersection(preds))
TN = len(huc12.HUC_12.unique())-len(preds)
FN = len(set(true_vals).intersection(set(huc12.HUC_12.unique()) ^ set(preds)))

#precision = TP/(TP+FP)
#recall = TP/(TP+FN)
#acc = (TP+TN)/(TP+TN+FP+FN)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [06:25<00:00, 96.37s/it]


In [79]:
print([TP, FP, TN, FN])

[0, 0, 2382, 106]


Let's try something other than predicting locations

In [79]:
path = r'J:\\Geology\\WSGS\\GIS\\Predicted locations\\'
allFiles=glob.glob(path+"*.shp")
points = gpd.read_file(r'J:\Geology\WSGS\GIS\usgs_locations.shp')
points= points.to_crs({'init': 'epsg:3732'})

validations = []
for shapefile in allFiles:
    shape = gpd.read_file(shapefile)
    shape = shape.to_crs({'init': 'epsg:3732'})
    chosen = gpd.sjoin(points, shape)
    chosen['element'] = shapefile[-10:-4]
    validations.append(chosen)

In [95]:
selectedSamples = pd.concat(validations)

In [85]:
selectedSamples.to_file(driver='ESRI Shapefile', filename='usgs_samples_clipped.shp')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [149]:
selectedSamples.iloc[0].element[0:2]

'ag'

In [ ]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\ngdbrock-fUS56\\ngdbrock-fUS56\\'
allFiles=glob.glob(path+"*.csv")
rocks=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file)
    listed.append(df)
rocks = pd.concat(listed, axis=1)

C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,10,14,16,23,25,41,63,65,67,71,73,83,85,97,105,109,113,119,133,145,149,163,177,183) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (16,34,52,86,114,124,184) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
rocks[rocks['lab_id'] == selectedSamples.iloc[50].lab_id].filter(like=selectedSamples.iloc[50].element[0:2])
